<a href="https://colab.research.google.com/github/kotharisanjana/CMPE258_DeepLearning_Spring2023/blob/main/Assignment_6/DL_Pipeline_with_Custom_Components.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colab demonstrating custom components in neural networks sing tensorflow

In [1]:
# Imports

import math
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
# Ingest data

dataset, info = tfds.load('cats_vs_dogs', with_info=True)
dataset = dataset['train'].take(500)

In [3]:
# Preprocess images

def preprocess_image(sample):
    image = sample['image']
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (150, 150))
    image = tf.reshape(image, (1, 150, 150, 3))
    label = sample['label']
    return {'image': image, 'label': label}

dataset = dataset.map(preprocess_image)

In [4]:
# Split the dataset into training and testing sets

train_dataset_ = dataset.take(400)
test_dataset_ = dataset.skip(400)

train_dataset = train_dataset_.map(lambda x: (x['image'], tf.expand_dims(x['label'], axis=-1)))
test_dataset = train_dataset_.map(lambda x: (x['image'], tf.expand_dims(x['label'], axis=-1)))

Custom loss function

In [5]:
def huber_loss(y_true, y_pred):
  y_true = tf.cast(y_true, tf.float32)
  error = y_true - y_pred
  is_small_error = tf.abs(error) < 1
  squared_loss = tf.square(error) / 2
  linear_loss  = tf.abs(error) - 0.5
  return tf.where(is_small_error, squared_loss, linear_loss)

Custom initilializer

In [6]:
def glorot_initializer(shape, dtype=tf.float32):
  stddev = tf.sqrt(2. / (shape[0] + shape[1]))
  return tf.random.normal(shape, stddev=stddev, dtype=dtype)

Saving and loading model with custom object

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=glorot_initializer)
])

model.compile(optimizer='adam', loss=huber_loss, metrics=['accuracy'])

model.fit(train_dataset, epochs=1, batch_size=16, validation_data=test_dataset)

400/400 [==============================] - 65s 156ms/step - loss: 0.2652 - accuracy: 0.4700 - val_loss: 0.2650 - val_accuracy: 0.4700


In [8]:
# Saving models with custom objects

model.save('model_with_custom_loss_initializer')

In [9]:
# Load saved model and run again

model = tf.keras.models.load_model('model_with_custom_loss_initializer', custom_objects={'huber_loss':huber_loss, 'glorot_initializer':glorot_initializer})
model.fit(train_dataset, epochs=1, batch_size=16)

400/400 [==============================] - 48s 118ms/step - loss: 0.2650 - accuracy: 0.4700


Saving a model with custom loss function and custom initializer and loading it again works correctly.

Custom regularizer

In [10]:
class Regularizer(tf.keras.regularizers.Regularizer):
  def __init__(self, factor):
      self.factor = factor

  def __call__(self, weights):
      return tf.reduce_sum(tf.abs(self.factor * weights))

Custom seed

Seed is set using random number generator for reproducibility so that the model produces the same results every time the model is run with the same inputs.

In [11]:
tf.random.set_seed(112)

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=Regularizer(0.001),)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_dataset, epochs=1, batch_size=16, validation_data=test_dataset)

400/400 [==============================] - 56s 138ms/step - loss: 0.7918 - accuracy: 0.5425 - val_loss: 0.6952 - val_accuracy: 0.5325


Adding custom regularizer helped improve training and validation accuracy

Custom activation and custom kernel constraint

In [13]:
def activation(z):
  return tf.math.log(1.0 + tf.exp(z))/tf.exp(z)

In [14]:
def kernel_constraint(weights):
  return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation=activation, kernel_constraint=kernel_constraint)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_dataset, epochs=1, batch_size=16, validation_data=test_dataset)

400/400 [==============================] - 49s 121ms/step - loss: 0.7808 - accuracy: 0.4500 - val_loss: 0.7738 - val_accuracy: 0.4700


Adding this custom kernel constraint made the loss nan which could be due to vanishing or exploding gradients. 

Custom learning rate scheduler


Updates learning rate at the beginning of each batch - increase the learning rate linearly during about half of training, then reduce it linearly back to the initial learning rate, and lastly reduce it down to close to zero linearly for the very last part of training.

In [16]:
K = tf.keras.backend

class LearningRateScheduler(tf.keras.callbacks.Callback):
    def __init__(self, iterations, max_lr=1e-3, start_lr=None, last_iterations=None, last_lr=None):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr,
                                   self.max_lr)
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                   self.max_lr, self.start_lr)
        else:
            lr = self._interpolate(2 * self.half_iteration, self.iterations,
                                   self.start_lr, self.last_lr)
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, lr)

In [17]:
learning_rate_schedule = LearningRateScheduler(math.ceil(len(train_dataset) / 16) * 2, max_lr=0.001)

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_dataset, epochs=2, batch_size=16, validation_data=test_dataset, callbacks=[learning_rate_schedule])

Epoch 1/2
400/400 [==============================] - 43s 104ms/step - loss: 298371168.0000 - accuracy: 0.5600 - val_loss: 3712411904.0000 - val_accuracy: 0.5300
Epoch 2/2
400/400 [==============================] - 40s 100ms/step - loss: 400102948864.0000 - accuracy: 0.5300 - val_loss: 2000104718336.0000 - val_accuracy: 0.5300


Custom callback

We define a logging callback that generates logs during start and end of training, testing and evaluation

In [19]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        keys = list(logs.keys())
        print('Starting training; got log keys: {}'.format(keys))

    def on_train_end(self, logs=None):
        keys = list(logs.keys())
        print('Stop training; got log keys: {}'.format(keys))

    def on_test_begin(self, logs=None):
        keys = list(logs.keys())
        print('Start testing; got log keys: {}'.format(keys))

    def on_test_end(self, logs=None):
        keys = list(logs.keys())
        print('Stop testing; got log keys: {}'.format(keys))

    def on_predict_begin(self, logs=None):
        keys = list(logs.keys())
        print('Start predicting; got log keys: {}'.format(keys))

    def on_predict_end(self, logs=None):
        keys = list(logs.keys())
        print('Stop predicting; got log keys: {}'.format(keys))

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_dataset, epochs=2, batch_size=16, validation_data=test_dataset, callbacks=[CustomCallback()])

Starting training; got log keys: []
Epoch 1/2
400/400 [==============================] - ETA: 0s - loss: 0.8525 - accuracy: 0.5475Start testing; got log keys: []
Stop testing; got log keys: ['loss', 'accuracy']
400/400 [==============================] - 46s 112ms/step - loss: 0.8525 - accuracy: 0.5475 - val_loss: 0.6823 - val_accuracy: 0.5625
Epoch 2/2
400/400 [==============================] - ETA: 0s - loss: 0.7822 - accuracy: 0.6650Start testing; got log keys: []
Stop testing; got log keys: ['loss', 'accuracy']
400/400 [==============================] - 39s 98ms/step - loss: 0.7822 - accuracy: 0.6650 - val_loss: 0.6560 - val_accuracy: 0.8025
Stop training; got log keys: ['loss', 'accuracy', 'val_loss', 'val_accuracy']


Custom layer

In [21]:
df = fetch_california_housing()

X_train, X_test, y_train, y_test = train_test_split(df.data, df.target.reshape(-1, 1), random_state=42)

sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

In [22]:
class CustomDense(tf.keras.layers.Layer):
    def __init__(self, units, activation='relu'):
      super().__init__()
      self.units = units
      self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
      self.kernel = self.add_weight(shape=(input_shape[1], self.units), initializer='normal', trainable=True)
      self.biases = self.add_weight(shape=(self.units,), initializer='zeros', trainable=True)
      super().build(input_shape)

    def call(self, X):
      return self.activation(X @ self.kernel + self.biases)

Custom model

In [23]:
class CustomModel(tf.keras.Model):
    def __init__(self, **kwargs):
      super().__init__(**kwargs)
      self.hidden1 = CustomDense(100)
      self.hidden2 = CustomDense(50)
      self.hidden3 = CustomDense(10)
      self.output_ = CustomDense(1)

    def call(self, input):
      hidden1 = self.hidden1(input)
      hidden2 = self.hidden2(hidden1)
      hidden3 = self.hidden3(hidden2)
      concat = tf.keras.layers.concatenate([input, hidden3])
      output = self.output_(concat)
      return output

model = CustomModel()

In [24]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_scaled, y_train, epochs=2, validation_data=(X_test_scaled, y_test))

Epoch 1/2
484/484 [==============================] - 3s 4ms/step - loss: 0.9764 - accuracy: 0.0028 - val_loss: 0.4536 - val_accuracy: 0.0021
Epoch 2/2
484/484 [==============================] - 2s 3ms/step - loss: 0.4331 - accuracy: 0.0032 - val_loss: 0.4172 - val_accuracy: 0.0021


Custom gradient

In [25]:
@tf.custom_gradient
def custom_gradient_fn(z):
  def custom_gradient_fn_gradients(grads):  
      return grads * (1 - 1 / (1 + tf.exp(z))) 

  result = tf.math.log(1 + tf.exp(-tf.abs(z))) + tf.maximum(0., z)
  return result, custom_gradient_fn_gradients

x = tf.Variable([1000.])
with tf.GradientTape() as tape:
    z = custom_gradient_fn(x)

z, tape.gradient(z, [x])

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1000.], dtype=float32)>,
 [<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>])

Custom training loop

In [26]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [27]:
epochs = 2
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_fn = tf.keras.losses.binary_crossentropy
metrics = [tf.keras.metrics.Accuracy()]

In [28]:
for epoch in range(epochs):
  print('epoch =', epoch)
  for step, (X_batch, y_batch) in enumerate(train_dataset):
    with tf.GradientTape() as tape:
      y_pred = model(X_batch, training=True)
      y_batch = tf.reshape(y_batch, (1, 1))
      loss = loss_fn(y_batch, y_pred)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    for metric in metrics:
        metric(y_batch, y_pred)

epoch = 0
epoch = 1


Custom optimizer

Modifying Adam optimizer's behavior

In [29]:
class CustomOptimizer(tf.keras.optimizers.Adam):
  def __init__(self, dropout_rate):
    super().__init__()
    self.dropout_rate = dropout_rate
  
  def _resource_apply_dense(self, grad, var):
    dropout_mask = tf.keras.backend.random_binomial(tf.shape(var), p=1-self.dropout_rate)
    grad = grad * dropout_mask
    return super()._resource_apply_dense(grad, var)

In [30]:
model.compile(optimizer=CustomOptimizer(dropout_rate=0.1), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_dataset, epochs=2, batch_size=16, validation_data=test_dataset)

Epoch 1/2
400/400 [==============================] - 49s 121ms/step - loss: 6.9027 - accuracy: 0.5300 - val_loss: 0.6940 - val_accuracy: 0.5300
Epoch 2/2
400/400 [==============================] - 48s 120ms/step - loss: 0.6923 - accuracy: 0.5300 - val_loss: 0.6915 - val_accuracy: 0.5300


Custom metrics

In [31]:
def huber_metric(threshold):
  def huber_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    error = y_true - y_pred
    is_small_error = tf.abs(error) < threshold
    squared_loss = tf.square(error) / 2
    linear_loss  = threshold * tf.abs(error) - threshold ** 2 / 2
    return tf.where(is_small_error, squared_loss, linear_loss)
  return huber_loss 

In [32]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[huber_metric(0.4)])

model.fit(train_dataset, epochs=2, batch_size=16, validation_data=test_dataset)

Epoch 1/2
400/400 [==============================] - 40s 98ms/step - loss: 0.7824 - huber_loss: 0.1211 - val_loss: 0.6812 - val_huber_loss: 0.1176
Epoch 2/2
400/400 [==============================] - 40s 99ms/step - loss: 0.7000 - huber_loss: 0.1167 - val_loss: 0.6432 - val_huber_loss: 0.1093


Custom dropout

In [33]:
def custom_dropout(x, p):
    mask = tf.cast(tf.random.uniform(shape=tf.shape(x)) >= p, dtype=x.dtype)
    return x * mask

In [34]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Lambda(lambda x: custom_dropout(x, p=0.3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Lambda(lambda x: custom_dropout(x, p=0.2)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_dataset, epochs=2, batch_size=16, validation_data=test_dataset)

Epoch 1/2
400/400 [==============================] - 41s 99ms/step - loss: 0.7908 - accuracy: 0.5550 - val_loss: 0.6813 - val_accuracy: 0.6175
Epoch 2/2
400/400 [==============================] - 44s 109ms/step - loss: 0.6951 - accuracy: 0.5675 - val_loss: 0.6706 - val_accuracy: 0.6375
